In [1]:
import torch
import torch.nn as nn
import math

def create_1d_rope_embeddings(seq_len, dim, device='cpu'):
    # Ensure the dimension is even
    assert dim % 2 == 0, "Dimension must be even for 1D RoPE"

    # Create position indices
    inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2).float() / dim))
    inv_freq = inv_freq.to(device)

    # Create position embeddings
    sinusoid_inp = torch.einsum("i,j->ij", torch.arange(seq_len, dtype=torch.float, device=device), inv_freq)

    # Create the RoPE embeddings
    rope_sin = torch.sin(sinusoid_inp).unsqueeze(1).permute(1, 0, 2)
    rope_cos = torch.cos(sinusoid_inp).unsqueeze(1).permute(1, 0, 2)

    return rope_sin, rope_cos

def apply_1d_rope(token_embeddings, rope_sin, rope_cos):
    # Ensure the token embeddings dimension is even
    assert token_embeddings.shape[-1] % 2 == 0, "Token embeddings dimension must be even for 1D RoPE"

    # Split the token embeddings into even and odd dimensions
    token_embeddings_even = token_embeddings[..., 0::2]
    token_embeddings_odd = token_embeddings[..., 1::2]

    # Apply the RoPE embeddings
    token_embeddings_even_rotated = token_embeddings_even * rope_cos - token_embeddings_odd * rope_sin
    token_embeddings_odd_rotated = token_embeddings_even * rope_sin + token_embeddings_odd * rope_cos

    # Concatenate the rotated embeddings
    token_embeddings_rotated = torch.cat((token_embeddings_even_rotated, token_embeddings_odd_rotated), dim=-1)
    return token_embeddings_rotated

In [2]:
# For demonstration, assume we have a list of texts
texts = [
    "Lorem ipsum odor amet, consectetuer adipiscing elit. Sem dis justo dapibus hendrerit porta. Consequat tincidunt adipiscing dapibus, montes eu nunc curabitur. Purus rutrum sit venenatis est aliquet semper litora vehicula. Porta porttitor ligula tristique ad risus nec vitae. Malesuada libero volutpat pretium at semper potenti purus pretium hendrerit. Fermentum arcu facilisi dapibus tellus nibh dui, hac placerat. Egestas eros suspendisse semper nibh porta leo amet.",
    "Molestie et fusce penatibus ad nisi eros. Maecenas habitant hendrerit praesent aptent convallis. Quam adipiscing feugiat conubia posuere dui dis. Tempus dapibus aliquet facilisi taciti maximus. Imperdiet lorem dolor est; hac dapibus vehicula. Class turpis varius natoque vitae nam vehicula sociosqu pellentesque. Semper habitant dictumst magnis inceptos odio elementum platea. Vitae lacinia donec praesent est, curabitur viverra etiam cursus. Nisl platea mauris dapibus mauris pellentesque, nisl lobortis elit.",
    "Aliquet ut mus cubilia ullamcorper ad ut tellus. Eros vel metus aliquam aenean suscipit ante eget? Ad lacus fermentum mus tempor congue imperdiet blandit himenaeos. Maecenas turpis penatibus consequat bibendum tincidunt non nibh. Placerat urna consequat; finibus eleifend hendrerit rhoncus velit id iaculis. Aliquet est ultricies donec interdum tellus scelerisque facilisis. Rhoncus senectus in quis finibus taciti; ornare dis. Habitasse imperdiet justo tristique massa fringilla natoque, odio sit. Dui nullam facilisis fusce sapien hendrerit sapien dis molestie."
    "Orci vulputate vestibulum taciti finibus non interdum. Amet lorem integer fermentum rutrum ornare orci. Cras mollis dapibus lobortis; vehicula pharetra duis auctor hendrerit cursus. Justo ultricies egestas id arcu enim posuere interdum. Condimentum eleifend dictum nec penatibus montes ridiculus tellus lacinia. Conubia montes duis neque enim nisl conubia. Tempus neque ultrices vel adipiscing gravida praesent aptent. Scelerisque sapien sociosqu suspendisse tortor habitasse eleifend. Vulputate conubia lectus augue, senectus maximus fermentum habitant. Nascetur hac cras eros donec magna feugiat quam netus malesuada."
    "Inceptos quisque placerat himenaeos facilisi primis. Nostra natoque sem penatibus lectus dolor. Arcu consequat aliquam sodales taciti aptent. Mattis nisi mauris himenaeos blandit lacus et? Odio elementum sagittis eu nisi aliquam maecenas vulputate. Non convallis himenaeos est nostra congue dignissim tempus feugiat. Enim integer tellus mattis velit fermentum massa. Tristique ad aliquet blandit elit ante dis varius viverra."
    "Iaculis nulla sagittis id ridiculus ad vestibulum; ante augue adipiscing? Tempus hendrerit orci curabitur, velit dapibus vulputate tincidunt. Faucibus varius scelerisque malesuada at quisque tristique urna. Nisi commodo consequat aenean facilisis viverra suspendisse lacinia finibus. Lorem torquent adipiscing tincidunt sed aenean. Maecenas finibus posuere eu parturient dapibus vehicula varius. Felis justo facilisis inceptos ex etiam turpis, risus sodales. Elit vel tempus habitant sodales arcu eros taciti at neque.",
    "Vulputate dis interdum tincidunt ultricies molestie sociosqu sapien. Porta eu porta fusce nam hendrerit mollis elit. Dignissim mauris est class eu integer. Litora sociosqu blandit diam feugiat, dictum mus taciti eget. Condimentum habitant nostra ultricies facilisis habitasse. Montes dapibus maximus sapien penatibus inceptos mollis. Nostra cursus conubia ad suscipit tellus sociosqu volutpat. Nam vitae varius montes curae ante lectus diam.",
    "Fermentum convallis potenti nam turpis gravida penatibus. Integer venenatis rutrum pretium donec hac. Lectus at malesuada velit curae vehicula netus vestibulum. Libero tincidunt id varius potenti diam accumsan. Eros litora urna fames, interdum luctus erat. Auctor fringilla malesuada commodo sociosqu maecenas. Risus convallis molestie posuere vel ante tristique facilisis ligula. Pulvinar parturient turpis mollis suspendisse ante blandit.",
    "Efficitur aptent ac viverra ullamcorper eget sollicitudin eget cursus. Erat bibendum penatibus praesent quam sollicitudin. Ante quam faucibus mus volutpat phasellus penatibus. Senectus scelerisque turpis mus feugiat viverra morbi sociosqu. Class nec placerat ex, a hac nec. Curabitur quis fermentum tincidunt neque augue lacinia. Aex conubia ex vivamus ligula proin sit. Bibendum habitasse cursus congue morbi platea. Taciti est finibus accumsan iaculis montes rutrum. Consequat vulputate odio conubia sociosqu imperdiet porta!",
    "Dictumst quisque tellus dui tellus velit purus eget venenatis. Nec consequat aliquet curae tincidunt quam nisi. Auctor placerat class; sem phasellus eu netus. Non sodales suscipit molestie, egestas dapibus hac. Natoque feugiat imperdiet venenatis parturient hendrerit scelerisque finibus. Lacinia cubilia turpis libero mus velit dui tellus parturient nulla. Aenean dapibus aliquet penatibus himenaeos elit turpis. Ridiculus pretium facilisis vulputate condimentum cursus sed. Taciti urna gravida pellentesque nostra elementum, nisi efficitur torquent rutrum. Facilisis mauris varius sollicitudin accumsan turpis efficitur efficitur aliquam orci.",
    "Ligula adipiscing phasellus condimentum malesuada iaculis sed laoreet eros auctor. Conubia libero dignissim feugiat at lorem curae elementum convallis. Curabitur leo habitasse posuere quisque tortor vehicula nibh ac. Adipiscing odio purus nullam nisl; fusce dapibus. Elit maximus laoreet libero mus consectetur purus augue suspendisse. Neque lobortis diam quam mus; natoque accumsan. Urna adipiscing sit consequat posuere orci rhoncus quis sem. Consequat proin curae eleifend mauris dignissim magna laoreet.",
    "Himenaeos euismod malesuada quisque diam nam tincidunt. Justo nam lacinia vulputate elit justo class mi augue. Mollis dignissim imperdiet himenaeos, pellentesque sit primis? Sed posuere dui amet nunc est. Euismod praesent ornare eros vivamus vel habitant leo. Natoque rutrum suscipit pulvinar torquent lacinia iaculis pretium eget? Accumsan praesent elit euismod orci pharetra interdum quisque ad. Convallis vitae rutrum blandit, non justo commodo condimentum nascetur.",
    "Sapien auctor ad dui amet quisque. Potenti velit praesent vel conubia imperdiet. Maximus finibus consequat sapien parturient rhoncus felis. Erat placerat nisi montes; erat hac egestas quis. Bibendum metus cubilia ultricies dapibus purus cras mauris pellentesque. Accumsan turpis purus massa magna aliquam elit. Sollicitudin tempor sed est nascetur dui vestibulum lacinia tempor. Eu class est nisi metus dictumst turpis.",
    "Ridiculus tempus blandit amet nunc himenaeos semper himenaeos? Imperdiet vivamus venenatis phasellus, duis tempus vulputate. Duis porta quisque nostra nulla accumsan facilisis congue? Lacus bibendum leo iaculis vulputate, cursus posuere et orci. Hendrerit risus eleifend varius torquent eros; ornare duis. Amet magna torquent euismod praesent justo nunc scelerisque aliquet scelerisque. Sed consectetur euismod praesent amet; primis consectetur montes feugiat pellentesque.",
    "Quis facilisis mi magnis; nam nisi vitae. Senectus feugiat amet mattis faucibus est lobortis natoque. Auctor molestie interdum euismod, lacinia iaculis torquent gravida. Porttitor diam magna integer vivamus pellentesque quam tempus. Pellentesque tempor interdum adipiscing rhoncus dui vivamus. Commodo habitasse suspendisse, blandit vestibulum placerat sed nec. Facilisis et tristique maecenas sollicitudin ac convallis congue ipsum. Turpis sem convallis primis ipsum morbi porta pellentesque et.",
    "Quis id eleifend luctus facilisis nec phasellus. Dis turpis condimentum, sodales primis luctus non potenti. Est massa fermentum interdum sapien etiam pharetra habitant. Facilisi porta dolor faucibus dis facilisi. Porta aliquam fermentum ad eu auctor blandit finibus luctus. Sed ac urna vestibulum velit habitant. Mus varius purus dui eu bibendum commodo vulputate; sed maximus. Condimentum magnis enim facilisi tellus amet dictumst. Bibendum risus porta phasellus natoque lacus vel iaculis euismod.",
    "Varius cubilia enim sodales a proin lacus sagittis. Lacinia aliquam dui eleifend urna erat. Sollicitudin lorem sodales vivamus, duis adipiscing in odio. Ultricies ante efficitur penatibus blandit in. Eget sollicitudin risus dolor elit aliquet placerat porttitor arcu ad. Tempus orci habitant suspendisse a nisi ac ad purus. Molestie mollis sociosqu sollicitudin vitae ipsum amet.",
    "Lacus diam risus primis taciti tempus risus odio sociosqu! Ad vehicula litora netus taciti; augue commodo tristique. Tellus platea integer torquent imperdiet ultrices tempor. Tempus montes malesuada etiam fermentum primis dui torquent litora. Maximus feugiat aptent natoque metus taciti et. Nulla urna netus ac diam nostra duis. Sagittis aliquet nunc placerat volutpat nisi egestas tincidunt. Montes laoreet venenatis et sagittis tincidunt aliquet accumsan.",
    "Iaculis nunc faucibus vehicula; sociosqu etiam nibh consectetur arcu. Metus mattis gravida parturient netus posuere id. Porttitor habitant turpis odio elementum tincidunt habitant. Primis augue curabitur convallis ac nec neque cubilia. Varius nostra elit orci platea facilisi montes orci. Neque nulla convallis orci aptent placerat volutpat. Dolor nullam scelerisque tellus et proin imperdiet efficitur donec.",
    "Molestie sodales class metus; sociosqu fringilla consequat. Auctor magnis urna ac volutpat; libero orci duis. Nullam platea parturient nascetur dictumst odio nunc viverra suscipit. Maecenas lobortis aliquet dolor commodo pellentesque cras. Eros vitae posuere nostra iaculis tortor est. Elementum mollis imperdiet nunc praesent iaculis sed mi. Felis sollicitudin lorem habitant magna fusce commodo enim euismod.",
    "Duis in eleifend feugiat fermentum ac. Molestie nostra malesuada maximus habitasse quisque ornare. Adipiscing faucibus proin taciti facilisi dolor primis molestie id. Praesent mi integer libero viverra egestas eget ac. Dui diam potenti ex ante dapibus morbi gravida. Dis mauris lorem congue arcu magnis ultrices inceptos augue. Quam ante hendrerit dignissim montes dapibus convallis fringilla fringilla. Cras feugiat congue, sollicitudin velit laoreet tempor. Vestibulum nam facilisis dictum primis sociosqu. Gravida commodo fusce eget primis praesent maecenas.",
    "At quis vulputate mollis nec euismod; mi diam. Leo metus condimentum erat platea faucibus gravida. Odio finibus posuere tortor nibh laoreet; tellus ad primis. Magna condimentum suscipit nam ultrices dignissim ac ac sodales iaculis. Vestibulum eu dapibus phasellus elementum risus lobortis accumsan. Integer platea habitant hendrerit elit consectetur finibus suspendisse.",
    "Cras integer nulla mauris porttitor erat ad tortor in sem. Felis eros quis nullam, tortor condimentum habitasse. Interdum vulputate class enim sapien vel volutpat felis egestas. Neque taciti cursus orci maecenas purus. Ac metus interdum risus sociosqu elementum lobortis rhoncus? Ipsum mollis tempus mollis fringilla dis habitant per. Egestas natoque mollis habitant fusce risus interdum litora aptent litora.",
    "Rutrum viverra viverra euismod duis vel, vivamus potenti facilisis. Blandit volutpat enim porta ligula sem. Semper senectus sit cursus sem vivamus elementum fermentum. Dolor nullam risus maecenas, cursus urna vestibulum. Eget convallis fermentum tristique adipiscing torquent congue mus. Malesuada aliquet hendrerit varius lectus magna hendrerit maecenas. Maximus dolor maximus risus habitant vehicula augue. Massa iaculis dui bibendum dui hendrerit libero senectus."
    "Fusce velit porttitor quam lacus libero orci, gravida lectus? Nam consectetur mi ante vestibulum id, ultrices sagittis. Dignissim imperdiet lectus vulputate neque condimentum rutrum imperdiet senectus dis. Sociosqu taciti ultrices porta amet inceptos donec; eros phasellus efficitur. Netus posuere maximus fusce cras tortor quis lobortis. Eget finibus fringilla porttitor tempor nunc inceptos. Aliquam class congue suscipit auctor egestas nibh cubilia nostra. Luctus nascetur sem purus nunc pulvinar class erat ultrices magna? Pharetra cubilia tempor curabitur dignissim himenaeos. Dapibus felis non id ac nibh maecenas ante lobortis auctor.",
    "Class finibus porttitor urna fames; at laoreet ut. At tempor rhoncus praesent feugiat himenaeos tincidunt metus pellentesque nostra. Donec feugiat placerat imperdiet viverra; integer faucibus cubilia. Nam platea a commodo in sagittis dis montes aptent. Suscipit sagittis netus nisl nibh amet; placerat potenti odio. Sem hendrerit magnis integer libero nullam donec. Fermentum curae ipsum purus purus inceptos, quis curae velit sed. Ac sem luctus ultrices; iaculis elit hendrerit aliquam. Morbi blandit posuere mauris sapien sodales cubilia.",
    "Suscipit eu curae rhoncus feugiat tellus ornare. Duis efficitur neque accumsan litora arcu nec lobortis. Purus quam interdum sollicitudin sit sodales commodo. Vitae nullam dolor, maecenas parturient vel urna per. Quam dictum pharetra bibendum fames lacinia cursus. Finibus lobortis placerat accumsan mattis non ante luctus imperdiet. Dui mauris rhoncus purus cubilia nulla fames eu parturient. Parturient nisl gravida facilisis convallis posuere duis sollicitudin. Curae molestie platea tempus odio justo lorem vestibulum vel.",
    "Nostra interdum phasellus lectus parturient congue. Ut ultricies ut maximus habitant sodales. Urna maximus fusce molestie bibendum, consectetur luctus litora sapien. Elit quam lobortis hac sollicitudin netus risus. Sed per parturient at mattis parturient laoreet volutpat lacinia. Lectus quis lobortis ac habitasse congue pretium. Sem natoque ex proin platea morbi purus suscipit vehicula. Volutpat justo quis; augue nostra luctus lacus fames. Sit quam dolor, risus pulvinar mus nibh.",
    "Nec blandit purus elementum vitae; finibus cubilia nibh. Tristique netus potenti quis cras sed; fames magna. Dictum venenatis velit sed dignissim bibendum accumsan iaculis. Quisque eget id maecenas ultrices sociosqu. Ultrices maecenas mi viverra, mi primis blandit porttitor scelerisque. Integer id adipiscing sit parturient erat conubia purus blandit. Posuere arcu mattis nullam tellus quisque eleifend facilisis est. Purus blandit aenean potenti integer habitant. Quisque parturient senectus volutpat accumsan sociosqu duis non.",
    "Risus venenatis sagittis cubilia mi donec sed commodo primis. Efficitur dapibus mi scelerisque eget viverra hac tristique. Cras feugiat platea nisi justo habitasse; habitasse cras viverra. Habitasse sociosqu pellentesque neque nulla vel pellentesque. Dis malesuada consectetur vitae eget vel vitae nulla lorem fames. Sollicitudin quam eros suspendisse sollicitudin per curabitur rhoncus eget? Sodales nisi vulputate velit aliquet; dignissim non sem. Luctus dapibus pulvinar tellus inceptos ex lorem risus purus.",
    "Semper mollis phasellus faucibus dis magna libero sem. Urna blandit congue sed cursus adipiscing fringilla gravida ex? Consequat diam hendrerit eleifend amet litora nam malesuada. Conubia accumsan erat duis porttitor malesuada pulvinar. Ligula velit ad eu suspendisse sollicitudin finibus pulvinar ullamcorper. Cursus ad viverra platea vivamus vestibulum senectus purus nullam. Auctor inceptos venenatis lacus et elementum. Vulputate elementum lectus efficitur sem; venenatis habitasse potenti potenti neque.",
    "Lectus porta dis natoque ligula; aliquet orci eleifend. Vel maximus dignissim leo turpis parturient imperdiet nisi proin. Ut a cursus conubia eros eleifend. Aptent consectetur porta pretium integer elementum. Eleifend magna elementum himenaeos natoque nec posuere inceptos habitasse lobortis. Fermentum ad sapien sit placerat pretium faucibus. Vivamus varius potenti cubilia habitant aenean diam. Ridiculus potenti nascetur at iaculis aptent pellentesque quis.",
    "Lorem enim suspendisse sagittis euismod nullam sodales. Mollis tortor conubia mus dapibus vehicula nulla curae est. Ornare efficitur metus cursus egestas volutpat enim! Aptent odio tortor aptent eu consectetur donec tempus. Adipiscing hendrerit habitasse ridiculus cubilia aptent hac varius ante. Ante accumsan consequat magna, tortor himenaeos habitasse lectus.",
    "Hendrerit tellus interdum ante, sagittis sed finibus sociosqu. Diam magna metus cubilia tempus odio; hac natoque. Massa vivamus magnis volutpat pulvinar ultrices sit sit curabitur. Fermentum facilisi vulputate amet lacinia tempus morbi penatibus adipiscing. Tempor tempus ornare adipiscing nostra facilisis mi cursus enim. Placerat consectetur dictum ac risus efficitur sociosqu. Placerat ipsum duis nec consequat maecenas interdum platea taciti.",
    "Neque sodales fermentum senectus eleifend auctor velit. Suspendisse quis commodo mauris vulputate in hendrerit ac nisl. Ac mollis mattis mattis orci sit parturient arcu vulputate. Vehicula aliquam aliquam et rhoncus tortor orci facilisis. Lectus malesuada tristique magna duis etiam. Ac placerat phasellus et suspendisse sodales ante varius libero elit. Varius ultricies mi gravida risus elit metus eu ac.",
    "Volutpat facilisis ipsum accumsan lobortis enim commodo purus. Suspendisse habitant ullamcorper donec vel commodo dignissim non senectus. Quisque quis nascetur urna odio ac dolor iaculis sollicitudin. Lorem vulputate consequat convallis consectetur eleifend ullamcorper mollis. Facilisis felis neque nisl duis sodales, pellentesque venenatis sollicitudin. Platea ad justo, aliquet amet montes aptent platea. Pellentesque cursus convallis tortor ipsum venenatis sollicitudin donec dis duis. Suscipit vulputate penatibus eros dis lacinia fusce ipsum sagittis class. Ipsum ut massa ullamcorper ullamcorper morbi; urna lacinia luctus semper.",
    "Adipiscing conubia ex; iaculis magna rhoncus nam ex faucibus. Donec lectus id convallis netus ornare sociosqu tristique proin convallis. Eros nulla consequat iaculis facilisi himenaeos porta himenaeos etiam parturient. Torquent aliquam bibendum pellentesque litora blandit quis. Cras quisque faucibus suscipit leo luctus ultricies. Integer enim pretium donec sollicitudin tincidunt ornare molestie purus. Habitant eu egestas id, ornare malesuada diam. Efficitur praesent vitae nunc tincidunt massa etiam sem risus mi.",
    "Fringilla proin quis nascetur iaculis nisl non quam taciti. Est nostra bibendum sodales, vulputate turpis consequat litora. Mus libero luctus cras primis lectus molestie etiam. Faucibus mollis mauris accumsan tellus dolor in. Turpis pellentesque dictum netus hac duis elit odio. Donec quam eu finibus, mauris nam porta. Suscipit iaculis porta euismod sapien interdum tristique odio fames. Viverra laoreet ultrices lorem maecenas consectetur commodo curae egestas. Tristique feugiat dictum morbi nascetur magna scelerisque ut.",
    "Torquent porttitor maximus conubia aptent taciti. Nascetur ac lobortis tempor dis ullamcorper facilisi ipsum facilisis. Mauris pulvinar quis dis tellus massa mi velit. Torquent auctor habitant at fusce convallis. Fames accumsan maecenas tempus urna lorem dictumst tellus magnis. Adipiscing taciti augue convallis sagittis quam proin.",
    "Rhoncus fermentum lacus primis a euismod penatibus sed ullamcorper. Pretium amet habitasse gravida eros torquent imperdiet volutpat venenatis. Consectetur lectus turpis condimentum auctor sodales quam ipsum class lacinia. Consequat taciti viverra eleifend hendrerit eleifend turpis inceptos platea. Purus lacinia eget faucibus himenaeos ipsum eleifend. Nostra leo arcu pretium efficitur ante porttitor sapien sed? Feugiat conubia adipiscing ornare dignissim erat torquent; curabitur facilisis.",
    "Scelerisque maximus vivamus dui pulvinar nostra. Suscipit commodo molestie sapien a justo tristique. Sollicitudin interdum semper potenti torquent mollis sit hendrerit rhoncus. Pretium tortor et lacus vulputate faucibus dignissim. Hac potenti venenatis pharetra risus morbi. Ac quam fames duis mauris feugiat pharetra. Eleifend facilisi eleifend litora elit vivamus dis. Nulla et arcu nibh augue cursus nascetur risus. Sollicitudin porttitor tempor praesent faucibus aptent leo. Ac rutrum ultricies quam mattis ad quam.",
    "Urna eget ullamcorper, ante curabitur eros tincidunt. Magna magna vel facilisis eu mauris neque. Semper sociosqu nec nulla curabitur dignissim eleifend urna. Maximus imperdiet leo facilisis consectetur natoque sapien eget ad. Efficitur orci mi lacinia urna fusce mattis. Integer leo nam parturient sit sociosqu sem. Nullam molestie aptent ex; massa finibus magna. Blandit nunc montes magna class, etiam justo nam.",
    "Donec lectus odio consectetur dignissim dignissim varius tempus. Montes tortor lectus rhoncus blandit primis proin cras. Quisque neque augue turpis risus adipiscing orci sagittis quisque. Sociosqu leo pellentesque per inceptos, vivamus tempor ad. Proin in maecenas nibh viverra quisque lacinia porttitor consequat. Leo sed ullamcorper ultrices convallis parturient dictum integer laoreet. Turpis fames urna auctor class praesent curabitur class pellentesque. Tempus facilisi adipiscing suspendisse vulputate bibendum urna. Quam varius tortor dictum etiam etiam. Senectus pellentesque donec fermentum adipiscing urna quis velit."
    "Dis augue sollicitudin proin feugiat pretium primis lacus. Odio eleifend augue pretium volutpat accumsan, parturient molestie cursus. Luctus litora nam fames ante cursus natoque integer. Duis enim placerat urna nam vulputate, consectetur torquent consectetur. Nostra convallis enim velit, nostra duis euismod sociosqu natoque proin. Donec efficitur facilisis lobortis curabitur mi. Mattis tempus cras libero cras nullam efficitur mi.",
    "Tempus habitant accumsan senectus luctus nunc diam morbi. Sit dictumst lorem ipsum urna magnis. Habitant scelerisque dui congue cursus, eget lectus. Tempor nunc mauris molestie eget ante venenatis curabitur suspendisse. Ac lacus tellus nostra magnis auctor pretium quam. Ad et massa netus ac ornare donec. Adipiscing curabitur a dignissim montes netus mi nisi. Ridiculus aliquam at; consectetur congue curabitur diam.",
    "Maximus aliquam dolor mollis felis ad rhoncus. Netus quis class proin integer est rhoncus. Leo pharetra eu semper elit libero sem dapibus. Habitant accumsan inceptos urna massa etiam mollis porta non. Molestie quisque mattis feugiat; sodales vel inceptos. Habitasse vivamus dapibus in efficitur nunc. Imperdiet eleifend rutrum orci; facilisi faucibus tristique. Ultrices mattis senectus nullam ad pulvinar egestas.",
    "Nisl cras nam tincidunt at ultrices. Hendrerit auctor augue, nisl tempor lobortis suscipit. Dolor rutrum quisque pharetra dis semper nostra duis. Quam diam commodo proin curabitur odio ut risus. Turpis quam sapien himenaeos class semper consectetur cras felis. Placerat placerat blandit sociosqu erat neque commodo tempus. Dui nostra amet inceptos tempor hendrerit habitasse venenatis? Habitant primis accumsan fermentum cras penatibus ut eleifend. Nisl nec lobortis sollicitudin accumsan duis tellus parturient mi.",
    "Pulvinar suscipit vivamus laoreet dis facilisis. Vitae integer tempus nunc malesuada cursus. Cubilia arcu sit lectus ultrices; curabitur quisque ornare consectetur a? Magnis cubilia ridiculus nisi dictum gravida. Turpis facilisis vivamus magna nisi molestie id. Phasellus cubilia interdum ad lorem tristique venenatis mus nibh. Aptent elit vulputate tempus tempus fringilla magnis enim ante.",
    "Nunc imperdiet habitasse arcu ad congue. Montes mi iaculis molestie dictum himenaeos, porttitor parturient vulputate. Vel odio nisi gravida, velit at primis pellentesque erat. Condimentum nunc felis platea morbi a praesent. Tempus class per habitasse praesent platea phasellus vulputate interdum. Non taciti pharetra elementum sagittis enim nunc. Tellus pellentesque sagittis imperdiet fermentum ornare.",
    "Volutpat posuere ac duis tellus accumsan massa mus. Risus arcu primis cras conubia tellus imperdiet aliquet class turpis. Ex lorem leo faucibus purus accumsan; curabitur condimentum. Elementum sem enim turpis penatibus mi maximus. Eu finibus justo posuere penatibus accumsan sit laoreet. Dapibus nullam rutrum cubilia donec imperdiet curae. Per eleifend cursus rhoncus venenatis nostra. Posuere dictumst ante sapien nullam neque ac molestie.",
    "Scelerisque in vehicula elit malesuada cubilia bibendum erat nibh. Amet non ultricies, platea nec faucibus class? Maximus mauris faucibus ullamcorper facilisi laoreet. Acommodo netus fusce ut per. Est eleifend praesent turpis egestas diam ullamcorper nam. Mauris scelerisque massa metus lorem pretium libero."
    "Varius tortor torquent nullam enim pellentesque curae. Mollis platea duis consequat faucibus fringilla. Purus adipiscing blandit himenaeos eu erat. Diam ipsum pretium lobortis euismod in integer. Nec habitasse quam; dui potenti ligula dolor ullamcorper porta donec. Platea nulla nulla diam torquent praesent ut sem. Vel nunc hac consectetur platea ac libero. Diam nisl maximus rutrum dapibus leo. Nostra per sed facilisis conubia posuere, volutpat ut efficitur. Id condimentum porttitor venenatis euismod curabitur.",
    "Accumsan habitasse porttitor lectus placerat integer vitae laoreet litora quam. Penatibus diam euismod mi varius lacinia sociosqu. Viverra orci laoreet venenatis cursus morbi amet iaculis eget orci. Iaculis cursus congue donec imperdiet penatibus aptent. Sapien primis quam primis lacus neque torquent suspendisse. Massa risus a ipsum mauris porta dictumst.",
    "Praesent vivamus rhoncus erat potenti cursus nulla. Ex eros eu morbi dignissim tempor bibendum. Elit magna laoreet nibh duis dictum aptent. Placerat mollis aenean augue quam class fringilla? Hendrerit potenti cras suspendisse sodales tempus leo. Blandit aptent consequat class lacinia ex. Himenaeos sociosqu sapien volutpat parturient velit laoreet venenatis maximus.",
    "Ad magnis etiam malesuada interdum orci cras consequat ex? Laoreet nisl cras fermentum orci donec. Euismod luctus consequat venenatis conubia augue eu dapibus bibendum lorem. Orci fames libero et quis integer bibendum. Feugiat pretium convallis quam sociosqu himenaeos. Varius iaculis tortor gravida habitant nostra, maximus facilisis? Turpis mi mollis ex aliquam finibus nam. Potenti metus nunc tellus sed facilisi.",
    "Odio metus sollicitudin laoreet nascetur curae magna. Inceptos ipsum velit senectus luctus; luctus vel. Mi maecenas sapien egestas primis ipsum sem ullamcorper. Pretium malesuada mus nam aliquam aliquet cras dictum proin nec. Parturient fames tristique vehicula scelerisque et. Penatibus porttitor lacus tellus interdum condimentum ridiculus parturient. Congue dui natoque sit tristique lacinia semper ut duis. Semper ipsum ornare neque et aliquam. Vitae lobortis praesent cursus iaculis quisque dolor mauris.",
    "Proin facilisi erat natoque magna hac vel fringilla. Et netus mus suscipit tortor ultrices! Auctor quam fusce vulputate feugiat ante. Faucibus class in ultricies ex tincidunt lobortis, tempus posuere. Eu nisi etiam himenaeos, tincidunt massa congue litora justo. Vitae sagittis aliquam ornare conubia ornare eu. Nibh congue fusce vulputate cubilia; in odio habitasse et. Egestas neque ridiculus ridiculus conubia eu lacinia sagittis metus.",
    "Arcu semper sagittis eu torquent netus imperdiet per. Curae sagittis maecenas sociosqu aptent varius cursus cras. Amet accumsan conubia ultrices id blandit! Placerat consequat bibendum lobortis molestie justo nec interdum sollicitudin. Dui lectus enim pharetra placerat nulla massa rhoncus. Scelerisque sem finibus lacinia dolor dui. Dignissim euismod sem; id faucibus maximus potenti tristique ante. Ad et curae pretium odio etiam penatibus. Primis arcu finibus lectus orci ridiculus.",
    "Nec dictum ullamcorper nostra; euismod nam maecenas justo erat. Facilisis aliquam finibus neque aenean proin sit id. Vel ligula mi ultricies dictum nostra scelerisque lacinia. Imperdiet sit curabitur erat nisl arcu semper. Condimentum quisque morbi montes blandit mauris rhoncus interdum scelerisque adipiscing. Odio magnis egestas cursus sed urna ipsum hac dis ipsum. Pretium adipiscing conubia nostra vestibulum arcu nullam class. Parturient mi ultrices metus libero pulvinar efficitur. Turpis bibendum vulputate nascetur, sem interdum pretium sociosqu.",
    "Risus id blandit praesent mattis rhoncus aliquet. In ullamcorper sagittis ex; molestie curae ultrices hac nisl. Bibendum lorem parturient maecenas rutrum, metus fames. Quis at condimentum elementum ligula penatibus mollis. Aliquet facilisi pharetra suspendisse tortor augue risus eleifend finibus proin? Non aptent maecenas vulputate per eros ultrices. Pellentesque habitant cras quam blandit sit non.",
    "Primis ornare per tincidunt, cras scelerisque parturient parturient penatibus. Senectus maximus cubilia venenatis quis sit integer natoque pulvinar rutrum? Montes enim quis tellus malesuada risus. Quam ullamcorper habitasse in curae elementum tincidunt. Eleifend eget habitant etiam blandit penatibus felis adipiscing porttitor. Libero non metus posuere nulla fames malesuada at ridiculus felis. At lectus eget morbi sit facilisi non dis sed. Consectetur porttitor blandit tempor pharetra, neque et phasellus.",
    "Fusce molestie iaculis hac aliquet hac dolor bibendum. Dignissim iaculis lobortis senectus dapibus magna aptent. Vivamus nam condimentum at lacinia id a leo viverra vestibulum. Blandit nisl ipsum vulputate fringilla sagittis pellentesque blandit. Ex primis himenaeos vivamus congue massa nostra velit. Ac eget dictum porta potenti ut libero eget. Class senectus sit purus bibendum nisi. Nam libero conubia euismod consectetur lorem adipiscing enim.",
    "Pharetra habitant risus eros tortor magnis suscipit. Rhoncus et sociosqu lectus velit felis elementum tincidunt lacinia maecenas. Platea blandit blandit duis tempor urna ex mattis sapien aptent. Suscipit convallis habitant nascetur ultricies torquent dignissim dolor ex. Convallis nam malesuada aliquam habitasse mollis placerat est. Netus phasellus libero tristique etiam convallis venenatis, nostra id.",
    "Congue orci quisque sed venenatis adipiscing sodales dictum placerat eget. At habitant lorem bibendum lacus blandit fames ipsum nulla. Ornare magna semper sagittis, varius non efficitur sollicitudin. Massa orci neque erat sit egestas montes enim. In massa dui ut nulla nisl hac sodales at neque. Inceptos ipsum consequat suspendisse ullamcorper rhoncus pulvinar. Ipsum ultricies molestie hendrerit sit vitae leo dis.",
    "Ultrices ac placerat magnis eros nibh elementum. Suscipit duis ut diam taciti sociosqu dui. Urna dignissim fames nostra interdum condimentum erat. Ipsum donec natoque id integer imperdiet consequat pulvinar sem. Potenti cubilia nostra; pellentesque elit odio dignissim commodo maecenas. Mi cubilia enim condimentum per torquent magna conubia porta. Etiam orci justo faucibus arcu mollis semper laoreet donec.",
    "Senectus taciti fringilla ante maecenas commodo scelerisque mus ipsum. In posuere hendrerit ligula sit tellus. Ex ultricies egestas risus, lorem mauris litora. Sem vel torquent porta tellus neque interdum dolor lacus. Nullam euismod tristique fames sociosqu magna. Dictum consequat et quam, fusce libero enim iaculis. Cras ultrices facilisis suscipit sit tempor ultrices sociosqu. Arcu ex congue himenaeos nisi hac elementum facilisi sapien.",
    "Elementum donec parturient consequat velit accumsan libero tincidunt. Et aliquet dui ullamcorper, ac eleifend pellentesque vivamus. Aliquam turpis eget eros fermentum pretium felis laoreet ornare. In magna sollicitudin blandit tincidunt nec malesuada. Hendrerit nam per tincidunt fringilla fringilla elit. Mauris nunc amet tincidunt, luctus aliquam eros ornare ridiculus. Etiam tincidunt habitasse consequat pulvinar risus cubilia lobortis euismod amet. Sed scelerisque libero purus accumsan posuere fermentum ullamcorper. Urna eros sollicitudin himenaeos bibendum pellentesque. Leo fringilla semper conubia dis platea integer laoreet eu suscipit.",
    "Interdum cras faucibus neque accumsan conubia diam eget. Justo adipiscing ut euismod class eget lobortis. Dapibus donec mi nec porta; maximus donec. Facilisis vel ullamcorper phasellus eros aliquet enim faucibus. Tempor turpis cras vel euismod ante. Fusce purus natoque lacinia eleifend odio. Dignissim cursus nam tortor porta turpis nisi.",
    "Praesent cursus faucibus natoque fames finibus a eget egestas. Aliquet accumsan magna hac porttitor praesent cubilia ullamcorper. Amet curae fermentum facilisis eu ante urna curabitur. Dignissim efficitur inceptos finibus ante venenatis venenatis vivamus lacinia. Posuere dictum posuere pulvinar, nisi fames fusce luctus. Libero semper montes pharetra hendrerit massa fringilla; odio sem ridiculus. Praesent elit congue ipsum sollicitudin lacinia a. Accumsan elementum maecenas tristique eget a suscipit nisl gravida.",
    "Dis dapibus non diam eros finibus ad imperdiet. Sagittis vivamus tellus; pretium luctus torquent hendrerit. Arcu mollis libero justo, varius sit felis penatibus. Nunc inceptos dictum, convallis non congue posuere morbi. Suscipit litora dapibus vulputate; semper fusce lacinia. Viverra magnis vulputate pretium phasellus odio viverra. Gravida mus taciti cras vivamus augue.",
    "Ligula consequat cubilia enim enim suspendisse phasellus ac efficitur. Magna commodo accumsan finibus proin hac vestibulum habitant. Massa inceptos mus phasellus sit curabitur platea? Condimentum ultricies sem finibus himenaeos pulvinar quis cursus nibh condimentum. Porta sed potenti eu potenti ad mollis pellentesque maximus. Mollis eleifend tempus varius potenti nec dis integer.",
    "Ex aliquam duis, quis vitae quam molestie. Aliquam dapibus dolor laoreet primis cursus odio penatibus. Elementum potenti eu dis feugiat; velit dignissim. Tincidunt nam vivamus pellentesque, suscipit dui ante. Arcu purus sit, sit gravida mollis consectetur varius justo amet. Placerat proin ornare mollis ridiculus eget a. Fringilla egestas urna aenean habitasse, aliquet natoque luctus. Enim nostra lorem rutrum in eleifend bibendum hendrerit.",
    "Orci a ullamcorper luctus vivamus sodales blandit eleifend nam. Nostra nascetur justo lacus, ultrices elit suspendisse. Varius non platea eu consectetur massa himenaeos. Varius nullam vestibulum taciti vivamus erat fusce ornare sit. Sed nisi nulla potenti, sagittis elementum condimentum euismod finibus. Et torquent scelerisque velit diam, dolor tristique eget efficitur. Posuere aptent imperdiet aliquam est congue ex purus faucibus magna. Tellus taciti vulputate mollis justo molestie donec. Suscipit fringilla litora porttitor aenean magnis ultricies class.",
    "Tortor sociosqu sed convallis mus etiam tincidunt curae. Interdum elementum odio vitae accumsan porttitor efficitur arcu. Senectus bibendum vel at auctor fames convallis arcu. Nullam ut sit elementum molestie orci velit penatibus. Dis elementum ex augue interdum nascetur finibus. Nascetur pharetra urna euismod eleifend; ultricies lobortis nostra. Condimentum fusce etiam habitant; facilisi vulputate lobortis. Molestie lobortis habitasse elementum nec orci. Sodales lorem bibendum molestie per adipiscing montes cubilia.",
    "Eleifend quis ac mauris ad curabitur fringilla. Fringilla massa eu taciti aliquam consequat maecenas lacus donec est. Primis nostra aptent at mattis primis convallis urna consectetur? Potenti habitant gravida ridiculus sapien torquent ad et quam fames. Quam morbi vehicula risus imperdiet dolor est. Efficitur massa condimentum habitant maecenas viverra!",
    "Mattis mauris curae platea potenti efficitur scelerisque ultrices parturient. Facilisis cubilia non, nisl egestas libero nostra. Malesuada primis consectetur lacinia proin porta inceptos netus mauris duis. Quis risus nunc ridiculus nostra faucibus aenean volutpat metus vulputate. Vestibulum efficitur auctor ligula phasellus fames suspendisse. Urna nisi sollicitudin purus class ultrices. Orci aenean rutrum turpis auctor luctus massa iaculis nisi. Vestibulum leo sed convallis senectus vulputate faucibus facilisi. Suspendisse vel sollicitudin dapibus lobortis lobortis himenaeos suspendisse lacus. Vestibulum porttitor class mus sodales tellus arcu erat.",
    "Dis libero nec accumsan massa scelerisque; cras augue diam. Nullam urna eleifend augue metus lobortis elit congue? Sem magnis odio tortor est porta ipsum; sagittis fermentum. Placerat conubia habitasse non dapibus semper accumsan magna vestibulum tellus. Curabitur fusce aliquam euismod quam libero purus luctus molestie. Mollis turpis proin senectus fringilla netus dapibus habitasse. Non curabitur ad elit porta mi fames malesuada. Eros mi tempor porttitor per quisque eleifend nam cras. Pharetra primis ligula pulvinar metus pretium.",
    "Viverra eleifend consectetur metus nascetur dolor scelerisque accumsan etiam. Cursus bibendum duis odio; nisl velit proin. Non enim laoreet sit; malesuada auctor torquent. Lacus nisi malesuada quam integer natoque dignissim. Duis consectetur pharetra libero faucibus ac volutpat ac. Dis condimentum tempus congue eros dignissim sollicitudin magnis auctor. Dignissim facilisis ligula mi morbi morbi et torquent a.",
    "Dolor ipsum euismod imperdiet taciti viverra eget suspendisse. Nam luctus platea, auctor sociosqu cras placerat suscipit. Nibh praesent pharetra pharetra torquent quisque. Leo primis a sociosqu curae himenaeos et habitant posuere. Et maximus ligula, nam nostra etiam varius? Pharetra nam rutrum eu fames, tincidunt semper.",
    "Faucibus porta ad euismod sem ad leo conubia. Vivamus porta dis bibendum hac mus sem. Hendrerit natoque non et montes efficitur vulputate orci. Urna maximus curae mus eleifend potenti leo. Hac tortor leo augue consequat pellentesque. Purus tristique semper ornare vulputate auctor molestie turpis donec. Proin habitant facilisis magnis praesent tincidunt conubia tristique. Nulla quam congue malesuada tortor efficitur egestas class nostra.",
    "Consectetur in tortor gravida fringilla senectus auctor lobortis sodales. Dictum et magnis proin massa imperdiet. Bibendum fermentum ex scelerisque lacinia posuere tempus integer ante et. Integer phasellus et augue ullamcorper sociosqu dignissim class turpis. Condimentum pretium ornare justo ultricies leo in. In dolor adipiscing vel mattis ante hendrerit ullamcorper. Non ipsum odio elementum donec libero lobortis suscipit elementum fusce.",
    "Sollicitudin gravida conubia himenaeos eleifend nascetur est ipsum? Lacinia posuere dui auctor adipiscing; curae facilisi dapibus. Suspendisse turpis cras ridiculus velit vivamus quis. Pellentesque curae vulputate pretium scelerisque et. Suspendisse aliquam gravida convallis proin taciti sollicitudin finibus. Congue tempus sodales nec ornare ultricies mollis. Eleifend porttitor elit fusce parturient porta sem. Hac montes dictum nisl pulvinar bibendum eget.",
    "Mi maecenas conubia etiam libero lectus pretium tincidunt semper. Praesent eros metus a luctus ut diam neque nunc luctus. Fringilla rutrum interdum ullamcorper fringilla habitasse aptent curabitur hendrerit elit. Sociosqu senectus duis netus senectus parturient pretium adipiscing sagittis. Curae varius lacinia nunc pulvinar arcu est dolor pellentesque. Porta torquent dictumst porttitor platea habitant feugiat. Lobortis penatibus tortor cursus, nisl dapibus nullam tincidunt. Faucibus egestas ligula augue parturient sed sagittis facilisi. Maecenas convallis ante adipiscing sollicitudin ipsum leo imperdiet mollis.",
    "Facilisi nascetur lacus efficitur tempor habitant eros commodo. Taciti phasellus ullamcorper turpis amet aenean. Vel tristique eleifend consectetur, eros dis mattis. Donec magnis nibh tortor felis; facilisi fames turpis malesuada. Efficitur eget himenaeos mattis; scelerisque mollis odio risus. Laoreet tincidunt class facilisi urna euismod scelerisque. Arcu auctor bibendum congue, nunc in vehicula rutrum aenean non.",
    "Fusce lacinia maximus ullamcorper montes massa. Congue tincidunt convallis, accumsan tellus placerat netus. Dictum sociosqu neque vitae pretium vel iaculis pharetra. Natoque nec euismod tristique sociosqu quam facilisi. Volutpat ad parturient sagittis auctor lacus justo sodales. Tempus commodo nisi maecenas neque luctus ridiculus hendrerit. Magna nunc integer fusce iaculis vel primis congue magna. Mus turpis tortor mi porttitor mi.",
    "Rhoncus fames pretium conubia luctus, purus varius eget mauris luctus. Dolor tellus platea erat enim aenean urna. Porttitor libero dui semper sagittis vehicula. Magnis quisque aliquam pretium aenean malesuada eleifend. Vel ligula torquent morbi mauris porta. Facilisis ullamcorper morbi vel aptent lorem.",
    "Tempor ex proin cras pulvinar cubilia proin felis. Donec dapibus hac viverra orci fringilla donec. Dis aptent auctor commodo venenatis, nostra eget. Parturient velit imperdiet ullamcorper; enim habitasse aenean. Justo penatibus ac conubia lacus egestas vivamus purus. Montes ligula nascetur dui nascetur consequat posuere aliquet. Lorem vivamus bibendum velit convallis dignissim placerat parturient fusce.",
    "Lorem ad duis platea consequat rutrum feugiat ex. Diam ullamcorper egestas vulputate urna ipsum egestas mattis aliquam. Cubilia nisi fusce faucibus elit porta sodales. Molestie scelerisque habitasse malesuada fames metus praesent praesent faucibus. Himenaeos curae ac maximus dui inceptos mauris hac interdum pellentesque. Urna ultricies consequat, nec hendrerit nulla senectus habitant himenaeos. Enim bibendum platea sit sagittis vestibulum eros; ligula rhoncus. Nec sollicitudin dolor, magnis aliquet viverra dis penatibus. Eget proin eleifend viverra mollis mattis penatibus.",
    "Turpis tempus habitasse dapibus odio fames suscipit. Porttitor scelerisque accumsan posuere odio taciti consequat? Aultricies ultricies quam tortor ipsum platea. Diam ac elementum ornare quis tellus metus tellus. Penatibus leo rhoncus varius pharetra conubia penatibus. Amet integer laoreet eleifend ut nulla pellentesque augue dapibus. Nibh sodales tincidunt quis vivamus tellus lacinia primis sollicitudin ornare. Id elit sed mattis rutrum penatibus.",
    "Efficitur nisi phasellus a mus tristique natoque et. Leo etiam sodales vestibulum luctus velit. Enim etiam volutpat blandit magnis praesent. Donec pellentesque ullamcorper turpis ex risus facilisis faucibus vivamus erat. Convallis justo sit conubia tristique mattis. Ac pharetra amet venenatis nullam vehicula dis facilisis ipsum sodales. Lacus orci netus nascetur ad cubilia aenean fames. Massa purus vehicula imperdiet eget duis ipsum? Lacus natoque habitant risus; massa cubilia ornare at.",
    "Laoreet consequat vulputate nulla elit nisl dictum phasellus scelerisque. Vitae maecenas magnis tortor neque aliquam. Porttitor scelerisque dictumst dignissim curae maecenas viverra potenti vestibulum. Ipsum nulla fermentum aptent semper pellentesque mattis potenti. Nulla eu arcu aptent bibendum, pellentesque varius orci. Feugiat velit libero pellentesque nunc integer donec himenaeos a. Leo dignissim at nisi euismod elementum efficitur; tempus praesent. Aproin hac lacus sapien; rhoncus risus dapibus. Lacus donec convallis in feugiat aliquam primis.",
    "Torquent penatibus fames eros parturient eu eros. Netus purus tortor nascetur netus hac nam dictum scelerisque integer. Mus pretium bibendum pharetra nostra, habitasse blandit gravida. Ac interdum imperdiet purus fermentum ultrices. Per habitasse blandit porttitor pretium enim aliquam. Congue nunc lacinia quisque maximus integer.",
    "Nibh vitae ac porttitor odio nullam class diam? Curabitur magnis vulputate porta pellentesque non habitant. Tristique cursus justo iaculis dolor diam aliquet ac ut. Sodales auctor ex vivamus sed elit fusce? Phasellus fermentum maximus consequat; sed natoque ligula. Sed venenatis integer tempus est vehicula ultricies finibus tortor mauris. Luctus turpis magna natoque proin maecenas ullamcorper.",
    "Vehicula facilisis netus fames egestas feugiat accumsan nibh ac suscipit. Sagittis conubia posuere congue vestibulum nostra quam nascetur himenaeos. Commodo taciti hac nullam; nullam augue orci erat. Rhoncus ipsum pretium volutpat platea accumsan. Lacus mi egestas, curae pulvinar dapibus nam augue. Eu tempor senectus urna praesent mi per aliquet gravida? Faucibus tellus neque convallis accumsan etiam consequat egestas. Proin fermentum dui curae posuere ut sapien mus. Potenti pulvinar habitasse dictum mollis montes. Sit tortor varius fusce auctor lobortis.",
    "Torquent nibh sagittis posuere pretium aenean proin egestas. Conubia accumsan penatibus maximus ullamcorper arcu nostra duis conubia. Hendrerit penatibus nunc vel, efficitur enim aenean. Ipsum purus nibh facilisis quam himenaeos dapibus felis convallis? Etiam tincidunt elit torquent nascetur augue sed. Fermentum tincidunt sagittis faucibus facilisi eleifend fermentum interdum arcu eu. Ultricies sollicitudin ut himenaeos vel nisl.",
    "Malesuada aliquam semper sed molestie urna. Parturient amet dapibus habitant nam malesuada. Natoque nascetur magnis ridiculus lobortis metus ultricies. Risus ultrices quisque lacinia conubia luctus nullam! Morbi cubilia ex enim venenatis nisi et. Diam aliquet habitant quam viverra platea. Tincidunt integer condimentum mollis imperdiet, faucibus tristique. Sociosqu tempor finibus etiam imperdiet vitae ex nisi.",
    "Diam quisque dolor molestie nostra montes quis pellentesque. Ridiculus est fermentum et eu; odio habitasse semper taciti. Turpis sodales mauris etiam justo pretium; sapien ligula eu. Phasellus consectetur in consectetur leo condimentum nunc id. Phasellus egestas at lobortis vestibulum nibh. Accumsan velit litora cursus dignissim massa ad nulla massa nisi. Eget etiam facilisi curae ullamcorper accumsan class scelerisque. Nec etiam primis varius viverra mauris a cras hendrerit tempor.",
    "Praesent tincidunt donec natoque gravida malesuada tincidunt venenatis. Venenatis nostra morbi a odio elit posuere mollis nec justo. Tincidunt eget nascetur blandit fames; curae consectetur. Tincidunt fringilla et risus id, netus ornare. Risus consequat a diam cursus tempor orci at varius. Dui cursus eget suspendisse varius purus ornare interdum potenti.",
    "Ultrices mus dictumst tempus purus nec iaculis id. Ligula efficitur dis fusce pretium platea tellus potenti enim? Interdum nunc justo ac nostra turpis amet taciti habitasse? Eu ac imperdiet parturient volutpat, faucibus maecenas. Parturient interdum aliquet aenean natoque, enim himenaeos accumsan imperdiet. Morbi neque aliquet elit facilisis, justo varius venenatis. Vitae sociosqu natoque nec nostra imperdiet lacus dis quis venenatis. Litora class arcu commodo nisl bibendum congue. Curae potenti magnis aliquet lacus tempor lectus etiam mollis. Ligula feugiat lacinia sociosqu ad quisque suscipit nam nullam sem.",
    "Auctor maecenas platea netus suscipit vitae at leo. Egestas est sodales egestas fermentum, elit rutrum. Tincidunt gravida laoreet quisque pellentesque eleifend quisque rutrum fames. Habitant mollis arcu tincidunt egestas tellus hac aenean. Augue euismod nullam class sapien tincidunt quam. Porta proin nisi maecenas praesent velit class felis. Interdum molestie maximus metus pulvinar auctor condimentum. Mus semper lobortis fermentum pretium sem natoque montes duis velit.",
    "Primis nisi augue purus mauris mollis nascetur. Fames nisl amet nec senectus nascetur quis blandit quis urna. Porttitor mattis volutpat gravida vulputate cras? Fames ligula taciti platea euismod litora. Cras egestas sit enim volutpat cursus sagittis dictum. Senectus volutpat bibendum nostra habitasse, faucibus neque. Ultricies laoreet ultrices ipsum magnis; sodales quam amet. Lacus malesuada venenatis vulputate tempus felis; pretium ex aenean.",
    "Torquent pretium donec hendrerit eu leo mi parturient cursus. Semper turpis sem rutrum egestas quis commodo. Rhoncus tempor dignissim mauris fames non sem; leo tempus. Mi hendrerit a nullam sit cursus aliquet praesent. Aporta maximus gravida porttitor purus. Etiam porttitor sagittis cursus interdum sed platea lacinia. Scelerisque velit lobortis urna; class sapien sit. Nisl ligula venenatis lectus parturient euismod nam dapibus. Dapibus pharetra neque finibus vestibulum tincidunt potenti urna tortor!",
    "Lacus ac rutrum volutpat iaculis molestie. Curae ipsum etiam eu gravida odio; condimentum dolor placerat pellentesque. Cursus iaculis integer tellus porttitor vitae. Curae justo phasellus conubia odio, consectetur eros et. Ad imperdiet euismod aenean himenaeos egestas. Iaculis convallis dignissim; curabitur tempor consequat cursus aliquet massa. Proin lacus sed nunc posuere vivamus cras eros. Dictum in aliquet dictum platea quis suscipit. Quisque montes sagittis ipsum eleifend eget ultricies feugiat.",
    "In odio montes parturient auctor tortor himenaeos fringilla habitasse. Dictumst quis vivamus duis quam suspendisse dapibus habitant primis a. Nascetur placerat dictum; pulvinar luctus cras sodales ipsum. Nam congue efficitur, arcu ante maximus lectus. Nisi ad facilisi ornare ex fermentum. Laoreet leo feugiat aptent; sapien pharetra augue. Tristique tempor curabitur, faucibus dictum facilisi dolor. Tristique montes ridiculus magnis ridiculus tristique curae aptent.",
    "Sem metus tristique penatibus nullam a, nostra magna vivamus. In dis scelerisque nisi himenaeos quisque. Lacus conubia mus iaculis scelerisque aliquet eget. Sapien duis imperdiet fusce maximus risus. Tristique felis congue felis scelerisque sodales. Dolor purus tristique nam habitant velit molestie dictum. Velit hendrerit suscipit nisl lacinia curabitur penatibus. Facilisi dapibus pellentesque, inceptos aptent proin praesent auctor. Vel conubia augue eros accumsan mi adipiscing. Aliquam est etiam eleifend ultricies montes, eros est.",
    "Luctus cursus purus bibendum aenean cubilia fusce velit convallis. Porta pharetra augue sem parturient in natoque ad. Risus erat curabitur congue tempor cubilia adipiscing? Placerat ac sem elit congue tincidunt efficitur finibus porttitor pharetra. Maximus mus litora libero vivamus tempor. Rhoncus malesuada fusce id ante proin, sed luctus donec.",
    "Suscipit sem euismod himenaeos rhoncus vitae. Feugiat taciti adipiscing lobortis libero taciti maecenas sapien hac. Curae metus praesent magna habitasse facilisi id purus. Tristique lacinia nullam eros magnis turpis. Diam habitasse mollis praesent nibh; vestibulum habitasse a. Condimentum accumsan rutrum blandit etiam augue a elementum pharetra. Pretium orci porta aptent, hendrerit sit consectetur nam. Fringilla vehicula pharetra ligula penatibus efficitur natoque ex. Vehicula cras massa sollicitudin id natoque scelerisque consequat.",
    "Nullam gravida accumsan dignissim inceptos vitae suscipit mi feugiat. Dis feugiat eget sodales ligula urna sodales vel. Varius metus sodales massa non; suspendisse quis laoreet. Feugiat pretium elit malesuada ex vehicula. Purus velit dapibus venenatis nec eget cursus pharetra. Parturient imperdiet integer sodales, leo orci commodo auctor. Est leo rhoncus eu nunc arcu. Eget proin arcu aliquam vel non est turpis? Suspendisse blandit aptent cubilia felis mi euismod interdum.",
    "Consequat vivamus elit dis; integer penatibus risus. Torquent nisl imperdiet ut lobortis non. Integer rhoncus id dapibus, adipiscing ipsum penatibus arcu risus. Egestas ornare nostra; enim primis at tempus. Pulvinar massa facilisi vehicula vehicula ad mauris quisque! Blandit viverra suscipit vel dui urna cursus ullamcorper; libero ultrices. Lacus pretium adipiscing integer quisque turpis."
]

print(len(texts))

103


In [3]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# Train a BPE tokenizer
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(vocab_size=4000, special_tokens=["[UNK]", "[PAD]", "[EOS]"])
tokenizer.train_from_iterator(texts, trainer)

# Save and load the tokenizer
tokenizer.save("bpe_tokenizer.json")
tokenizer = Tokenizer.from_file("bpe_tokenizer.json")

In [4]:
import torch.nn as nn

class RMSNorm(nn.Module):
    def __init__(self, dim, eps=1e-5):
        super().__init__()
        self.scale = torch.nn.Parameter(torch.ones(dim))
        self.eps = eps

    def forward(self, x):
        norm = torch.norm(x, dim=-1, keepdim=True)
        return x / (norm + self.eps) * self.scale

In [5]:
import torch
import torch.nn as nn

class GeGLU(nn.Module):
    def __init__(self, dim_in, dim_out):
        super(GeGLU, self).__init__()
        self.proj = nn.Linear(dim_in, dim_out * 2)

    def forward(self, x):
        x, gate = self.proj(x).chunk(2, dim=-1)
        return x * torch.sigmoid(gate)

In [6]:
import torch.nn.functional as F

class GroupedAttentionLayer(nn.Module):
    def __init__(self, dim, num_heads, num_groups, max_seq_len, device='cpu'):
        super(GroupedAttentionLayer, self).__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.num_groups = num_groups
        self.head_dim = dim // num_heads
        self.max_seq_len = max_seq_len
        self.device = device

        # Linear projections for Q, K, V
        self.q_linear = nn.Linear(dim, dim)
        self.k_linear = nn.Linear(dim, dim)
        self.v_linear = nn.Linear(dim, dim)
        self.out_linear = nn.Linear(dim, dim)

    def forward(self, x):
        batch_size, seq_len, dim = x.shape

        # Precompute the RoPE embeddings
        rope_sin, rope_cos = create_1d_rope_embeddings(seq_len, self.head_dim, self.device)

        # Linear projections
        q = self.q_linear(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_linear(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_linear(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        # Apply RoPE to Q, K, V
        q = apply_1d_rope(q, rope_sin, rope_cos)
        k = apply_1d_rope(k, rope_sin, rope_cos)
        v = apply_1d_rope(v, rope_sin, rope_cos)

        # Group the queries, keys, and values
        q = q.view(batch_size, self.num_heads, self.num_groups, seq_len, self.head_dim // self.num_groups)
        k = k.view(batch_size, self.num_heads, self.num_groups, seq_len, self.head_dim // self.num_groups)
        v = v.view(batch_size, self.num_heads, self.num_groups, seq_len, self.head_dim // self.num_groups)

        # Compute attention using scaled_dot_product_attention
        attn_output = []
        for group in range(self.num_groups):
            attn_output.append(F.scaled_dot_product_attention(
                q[:, :, group], k[:, :, group], v[:, :, group], attn_mask=None, dropout_p=0.0, is_causal=True
            ))
        attn_output = torch.cat(attn_output, dim=2)

        # Reshape the output
        out = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, dim)

        # Linear projection
        out = self.out_linear(out)

        return out

class TransformerLayer(nn.Module):
    def __init__(self, dim, num_heads, num_groups, max_seq_len, device='cpu'):
        super(TransformerLayer, self).__init__()
        self.attention = GroupedAttentionLayer(dim, num_heads, num_groups, max_seq_len, device)
        self.norm1 = RMSNorm(dim)
        self.norm2 = RMSNorm(dim)
        self.ffn = nn.Sequential(
            nn.Linear(dim, dim * 4),
            GeGLU(dim * 4, dim * 4),
            nn.Linear(dim * 4, dim)
        )

    def forward(self, x):
        # Apply attention and add & norm
        x = self.norm1(x + self.attention(x))
        # Apply feed-forward network and add & norm
        x = self.norm2(x + self.ffn(x))
        return x

class TransformerModel(nn.Module):
    def __init__(self, num_layers, dim, num_heads, num_groups, max_seq_len, vocab_size, device='cpu'):
        super(TransformerModel, self).__init__()
        self.token_embedding = nn.Embedding(vocab_size, dim)
        self.layers = nn.ModuleList([
            TransformerLayer(dim, num_heads, num_groups, max_seq_len, device)
            for _ in range(num_layers)
        ])
        self.ln_f = RMSNorm(dim)
        self.head = nn.Linear(dim, vocab_size)

    def forward(self, x):
        # Token embeddings
        x = self.token_embedding(x)
        # Apply transformer layers
        for layer in self.layers:
            x = layer(x)
        # Final layer norm
        x = self.ln_f(x)
        # Output logits
        logits = self.head(x)
        return logits

In [7]:
import torch

# Tokenize the texts and ensure tensors are of type torch.long
encoded = tokenizer.encode_batch(texts)
sequences = [torch.tensor(enc.ids, dtype=torch.long) for enc in encoded]

# Pad sequences to the same length with explicit dtype
pad_token_id = tokenizer.token_to_id("[PAD]")
seq_len = max(len(seq) for seq in sequences)
sequences_padded = [torch.cat([seq, torch.tensor([pad_token_id] * (seq_len - len(seq)), dtype=torch.long)]) for seq in sequences]

# Create input and target tensors with dtype=torch.long
inputs = torch.stack(sequences_padded)
targets = torch.stack([torch.cat([seq[1:], torch.tensor([pad_token_id], dtype=torch.long)]) for seq in sequences_padded])

dataset = torch.utils.data.TensorDataset(inputs, targets)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)

In [8]:
print(tokenizer.decode(inputs[5].squeeze().tolist(), skip_special_tokens=False))
print(tokenizer.decode(targets[5].squeeze().tolist(), skip_special_tokens=False))

Efficitur aptent ac viverra ullamcorper eget sollicitudin eget cursus . Erat bibendum penatibus praesent quam sollicitudin . Ante quam faucibus mus volutpat phasellus penatibus . Senectus scelerisque turpis mus feugiat viverra morbi sociosqu . Class nec placerat ex , a hac nec . Curabitur quis fermentum tincidunt neque augue lacinia . Aex conubia ex vivamus ligula proin sit . Bibendum habitasse cursus congue morbi platea . Taciti est finibus accumsan iaculis montes rutrum . Consequat vulputate odio conubia sociosqu imperdiet porta ! [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [9]:
# Training Loop
model = TransformerModel(num_layers=8, dim=128, num_heads=8, num_groups=4, max_seq_len=seq_len, vocab_size=tokenizer.get_vocab_size(), device='cuda')
model.to('cuda')
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-3, weight_decay=0.01)
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.token_to_id("[PAD]"))

# In the training loop, ensure inputs and targets are on the correct device
for epoch in range(1000):
    total_loss = 0
    for batch in dataloader:
        input_batch, target_batch = batch
        input_batch = input_batch.to('cuda', dtype=torch.long)
        target_batch = target_batch.to('cuda', dtype=torch.long)
        logits = model(input_batch)
        loss = loss_fn(logits.view(-1, tokenizer.get_vocab_size()), target_batch.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")

Epoch 1, Loss: 6.6916351318359375
Epoch 2, Loss: 6.6184983253479
Epoch 3, Loss: 6.579282760620117
Epoch 4, Loss: 6.54323148727417
Epoch 5, Loss: 6.506240367889404
Epoch 6, Loss: 6.470029830932617
Epoch 7, Loss: 6.4330925941467285
Epoch 8, Loss: 6.396302700042725
Epoch 9, Loss: 6.3595194816589355
Epoch 10, Loss: 6.323327541351318
Epoch 11, Loss: 6.2874345779418945
Epoch 12, Loss: 6.252066135406494
Epoch 13, Loss: 6.217215061187744
Epoch 14, Loss: 6.18304967880249
Epoch 15, Loss: 6.149740219116211
Epoch 16, Loss: 6.117217063903809
Epoch 17, Loss: 6.085685729980469
Epoch 18, Loss: 6.05506706237793
Epoch 19, Loss: 6.025468826293945
Epoch 20, Loss: 5.996875762939453
Epoch 21, Loss: 5.969330310821533
Epoch 22, Loss: 5.942866325378418
Epoch 23, Loss: 5.917479038238525
Epoch 24, Loss: 5.8932108879089355
Epoch 25, Loss: 5.869968414306641
Epoch 26, Loss: 5.847775459289551
Epoch 27, Loss: 5.82656717300415
Epoch 28, Loss: 5.806384563446045
Epoch 29, Loss: 5.787160396575928
Epoch 30, Loss: 5.768887

In [10]:
def infer(model, tokenizer, input_text, max_length=250, device='cuda'):
    # Tokenize the input text
    encoded = tokenizer.encode(input_text)
    input_ids = torch.tensor(encoded.ids, dtype=torch.long).unsqueeze(0).to(device)

    # Generate text
    generated_ids = input_ids
    with torch.no_grad():
        for _ in range(max_length):
            # Pass the input through the model
            outputs = model(generated_ids)
            # Get the last token logits
            next_token_logits = outputs[:, -1, :]
            # Sample the next token
            next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
            # Append the next token to the sequence
            generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)
            # Stop if the EOS token is generated
            if next_token_id.item() == tokenizer.token_to_id("[EOS]"):
                break

    # Decode the generated sequence
    decoded_text = tokenizer.decode(generated_ids.squeeze().tolist(), skip_special_tokens=True)
    return decoded_text

# Example usage
input_text = "Lorem ipsum"
generated_text = infer(model, tokenizer, input_text, max_length=50)
print(f"Generated text: {generated_text}")

Generated text: Lorem ipsum mauris eros magna . . nibh . . . malesuada non . erat cras sit . . nostra , semper montes etiam mollis semper . . . . . . convallis pellentesque . . . , feugiat mi magna aptent mauris . adipiscing lacinia pharetra magnis Placerat Dignissim . .
